In [1]:
import pandas as pd

In [2]:
from arcgis.gis import GIS
import arcgis.network as network

my_gis = GIS(username='GHRBotha', password='GBHyc3G5WqMU')
my_gis

GIS @ https://renierbotha.maps.arcgis.com version:7.1

In [20]:
map = my_gis.map("San Diego")

In [21]:
map

MapView(layout=Layout(height='400px', width='100%'))

In [12]:
map.basemaps# = 'street-night-vector'

['dark-gray',
 'dark-gray-vector',
 'gray',
 'gray-vector',
 'hybrid',
 'national-geographic',
 'oceans',
 'osm',
 'satellite',
 'streets',
 'streets-navigation-vector',
 'streets-night-vector',
 'streets-relief-vector',
 'streets-vector',
 'terrain',
 'topo',
 'topo-vector']

In [22]:
map.basemap = 'streets-night-vector'

In [23]:
rest_content = my_gis.content.search("San Diego Restaurants", item_type="Feature Layer", max_items=10)

In [24]:
rest_content

[]

In [24]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

'https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World'

In [25]:
cf_layer = network.ClosestFacilityLayer(analysis_url, gis=my_gis)

In [26]:
from arcgis import geocoding

In [27]:
hospitals_addresses = ['Estrada Adhemar Bebiano, 339 Del Castilho, Rio de Janeiro RJ, 21051-370, Brazil',
                       'R. José dos Reis Engenho de Dentro, Rio de Janeiro RJ, 20750-000, Brazil',
                       'R. Dezessete, s/n Maré, Rio de Janeiro RJ, 21042-010, Brazil',
                       'Rua Dr. Miguel Vieira Ferreira, 266 Ramos, Rio de Janeiro RJ, Brazil']


In [29]:
from arcgis.features import Feature, FeatureSet

In [30]:
hosp_feat_list = []

for address in hospitals_addresses:
    hit = geocoding.geocode(address)[0]
    hosp_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])

    hosp_feat_list.append(hosp_feat)

In [34]:
hospitals_fset = FeatureSet(features=hosp_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [35]:
map1 = my_gis.map('Rio de Janeiro, Brazil')
map1

MapView(layout=Layout(height='400px', width='100%'))

In [36]:
map1.draw(hospitals_fset, symbol={"type": "esriSMS",
                                  "style": "esriSMSSquare",
                                  "color": [76,115,0,255],
                                  "size": 8,})

In [37]:
incident_coords = '-43.281206,-22.865676'
reverse_geocode = geocoding.reverse_geocode({"x": incident_coords.split(',')[0], 
                                              "y": incident_coords.split(',')[1]})

incident_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])

In [38]:
incident_fset = FeatureSet([incident_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})

In [63]:
map1.draw(incident_fset, symbol={"type": "esriSMS",
                                 "style": "esriSMSCircle",
                                 "color": [255,0,0,255],
                                 "size": 8})

In [40]:
result = cf_layer.solve_closest_facility(incidents=incident_fset,
                                        facilities=hospitals_fset,
                                        default_target_facility_count=4,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])


In [41]:
result.keys()

dict_keys(['messages', 'routes', 'facilities'])

In [42]:

result['routes'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])

In [43]:
result['routes']['features'][0].keys()

dict_keys(['attributes', 'geometry'])

In [44]:
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)

In [45]:
routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])


In [46]:
map1.draw(routes_fset)